In [1]:
from geopy.geocoders import Nominatim
import pandas as pd
import folium
from folium.plugins import MarkerCluster

In [3]:
INFILE = 'Collaborators.xlsx'
OUTFILE = 'CollabMap.html'

Input_df = pd.read_excel(INFILE)
print(Input_df)


                               Institution             City    CountryName  \
0                   University of Limerick         Limerick        Ireland   
1      Memorial University of Newfoundland       St. John’s         Canada   
2    Computational Science Research Center          Beijing          China   
3                               IIT Trichy  Tiruchirappalli          India   
4                 FernUniversitat in Hagen            Hagen        Germany   
5                          Tufts Univerity          Medford  United States   
6                   Holy Name’s University          Oakland  United States   
7   Cooch Behar Panchanan Barma University      Cooch Behar          India   
8                   University of Zaragoza         Zaragoza          Spain   
9                     University of Cyprus          Nicosia         Cyprus   
10                          DIT University         Dehradun          India   
11                                 TU Wien           Vienna     

In [4]:
geolocator = Nominatim(user_agent="my-ap")

Home = geolocator.geocode("Galway, Ireland")

print(f"Starting from {[Home.longitude, Home.latitude]}")

Starting from [-9.0490601, 53.2744122]


In [5]:
Countries = Input_df['CountryName'].tolist()
Institutes = Input_df['Institution'].tolist()
Cities = Input_df['City'].tolist()

In [6]:
world_map= folium.Map(tiles="cartodbpositron", location=[Home.latitude, Home.longitude])
marker_cluster = MarkerCluster().add_to(world_map)
popup_text = "University of Galway"
folium.CircleMarker(location=[Home.latitude, Home.longitude],
                    tooltip=popup_text,popup=popup_text,
                    color="red", fill =True).add_to(marker_cluster)

In [7]:
for i in range(len(Cities)):
    city = Cities[i]
    inst = Institutes[i]
    country = Countries[i]
    place = inst+", "+city+", "+country
    location =  geolocator.geocode(place)
    if (location == None):
        print(f"*** Warning, could not find {place}. Trying {inst}")
        location =  geolocator.geocode(inst)
    if (location == None):
        print(f"*** Warning, could not find {inst}. Trying {city}")
        location =  geolocator.geocode(city)
    if (location == None):
        print(f"*** Warning, can't find {place} or {city} or {inst}")
        break
    lat = location.latitude
    lon = location.longitude
    print(f'{i}: Adding {inst} at ({lat},{lon})')
    popup_text = inst
    folium.CircleMarker(location = [lat,lon],  tooltip=popup_text,popup= popup_text, fill =True).add_to(marker_cluster)

0: Adding University of Limerick at (52.67325805,-8.569809653548548)
*** Warning, could not find Memorial University of Newfoundland, St. John’s, Canada. Trying Memorial University of Newfoundland
1: Adding Memorial University of Newfoundland at (47.5733341,-52.733306586080026)
2: Adding Computational Science Research Center at (40.0459225,116.2780923)
*** Warning, could not find IIT Trichy, Tiruchirappalli, India. Trying IIT Trichy
*** Warning, could not find IIT Trichy. Trying Tiruchirappalli
3: Adding IIT Trichy at (10.804973,78.6870296)
4: Adding FernUniversitat in Hagen at (51.3771294,7.495154028855863)
*** Warning, could not find Tufts Univerity, Medford, United States. Trying Tufts Univerity
*** Warning, could not find Tufts Univerity. Trying Medford
5: Adding Tufts Univerity at (42.4184296,-71.1061639)
*** Warning, could not find Holy Name’s University, Oakland, United States. Trying Holy Name’s University
*** Warning, could not find Holy Name’s University. Trying Oakland
6: Ad

In [8]:
world_map.fit_bounds(world_map.get_bounds(), padding=(30, 30))
world_map.save(outfile=OUTFILE)